<a href="https://colab.research.google.com/github/Denis0213/MASTER-THESIS/blob/main/MODEL_PROGRAM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Libraries import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#from missingpy import MissForest
#from sklearn.impute import SimpleImputer
from sklearn.model_selection import KFold,StratifiedKFold
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import NearMiss
from sklearn.tree import DecisionTreeClassifier
from imblearn.over_sampling import SMOTE
from imblearn.combine import SMOTETomek,SMOTEENN 
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import auc,precision_recall_curve
from sklearn.metrics import average_precision_score
from sklearn.multiclass import OneVsRestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import average_precision_score
plt.rcParams["axes.labelsize"] = 18 
import warnings
warnings.filterwarnings('ignore')

pd.options.display.max_columns = None
pd.options.display.max_rows = None
from IPython.display import display

In [ ]:
# Algorithme d'apprentissage du modèle
def Fit_Algo(X,y):
  # Research space
  hyperparam= {"max_depth": [3, None],
              "max_features":range(1,20),
              "min_samples_leaf": range(1, 10),
              "criterion": ["gini", "entropy"]}
  # Creating decision tree objet
  dtc=DecisionTreeClassifier(class_weight='balanced')
  #Creating SMOTE object
  smote=SMOTEENN (random_state=42)
  # Oversampling
  x_smote,y_smote=smote.fit_resample(X,y)
  # RandomizedSearchCv objet creation
  random=RandomizedSearchCV(dtc,hyperparam,cv=10,n_iter=100)
  # Fiting
  model=random.fit(x_smote,y_smote)
  return model

In [ ]:
# Algorithme d'évaluation du modèle-ESSAI
def Score_Algo(dataset):
  X=dataset.drop('class',axis=1)
  y=dataset['class']
  score1=[]
  score2=[]
  for i in range(0,10):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
    model=Fit_Algo(X_train,y_train)
    y_pred=model.predict(X_test)
    score1.append(roc_auc_score(y_test,y_pred))
    score2.append(average_precision_score(y_test,y_pred))
  return round(np.mean(score1),3),round(np.mean(score2),3)

In [ ]:
# Algorithme d'évaluation du modèle
def Score_Algo(dataset):
  X=dataset.drop('class',axis=1)
  y=dataset['class']
  scv=StratifiedKFold(n_splits=10,shuffle=True,random_state=1)
  score1=[]
  score2=[]
  for train_index,test_index in scv.split(X,y):
    X_train,X_test=X.iloc[train_index],X.iloc[test_index]
    y_train,y_test=y.iloc[train_index],y.iloc[test_index]
    model=Fit_Algo(X_train,y_train)
    y_pred=model.predict(X_test)
    score1.append(roc_auc_score(y_test,y_pred))
    score2.append(average_precision_score(y_test,y_pred))
  return round(np.mean(score1),3),round(np.mean(score2),3)

# GLASS 5

In [ ]:
#Data overview
from google.colab import files
import io
uploaded= files.upload()

Saving glass5.csv to glass5.csv


In [ ]:
columns = ["RI","Na","Mg","Al","Si","K","Ca","Ba","Fe","class"]
glass5=pd.read_csv(io.BytesIO(uploaded['glass5.csv']),names=columns,header=None)
glass5.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,negative
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,negative
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,negative
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,negative
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,negative


In [ ]:
# Encoding glass5 class label
lbl_encoder=LabelEncoder()
glass5[['class']]=lbl_encoder.fit_transform(glass5[['class']])
glass5.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,0
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,0
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,0
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,0
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,0


In [ ]:
# Évaluation glass5 
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(glass5)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.8
AUPR=  0.63


In [ ]:
# A view of the 10 values of AUC
for i in range(0,30):
  print(perf1[i])
print('---------------------')
'Moyenne des AUC: {}'.format(round(np.mean(perf1),3))

0.692
0.718
0.828
0.859
0.733
0.809
0.742
0.701
0.72
0.818
0.723
0.861
0.832
0.745
0.878
0.867
0.814
0.814
0.754
0.715
0.847
0.847
0.859
0.789
0.842
0.715
0.847
0.764
0.947
0.911
---------------------


'Moyenne des AUC: 0.8'

In [ ]:
# A view of the 10 values of AUC
for i in range(0,30):
  print(perf2[i])
print('---------------------')
'Moyenne des AUC: {}'.format(round(np.mean(perf2),3))

0.482
0.481
0.673
0.747
0.539
0.658
0.521
0.43
0.492
0.632
0.501
0.72
0.678
0.568
0.757
0.77
0.667
0.661
0.53
0.426
0.724
0.716
0.706
0.655
0.671
0.467
0.724
0.597
0.897
0.806
---------------------


'Moyenne des AUC: 0.63'

# YEAST 5

In [ ]:
from google.colab import files
import io
uploaded= files.upload()

Saving yeast5.csv to yeast5.csv


In [ ]:
columns = ["Mcg","Gvh","Alm","Mit","Erl","Pox","Vac","Nuc","class"]
yeast5=pd.read_csv(io.BytesIO(uploaded['yeast5.csv']),names=columns,header=None)
yeast5.head()

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,class
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,negative
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,negative
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,negative
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,negative
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,negative


In [ ]:
lbl_encoder=LabelEncoder()
yeast5[['class']]=lbl_encoder.fit_transform(yeast5[['class']])
yeast5.head()

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,class
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,0
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,0
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,0
3,0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,0
4,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,0


In [ ]:
# Évaluation glass5 
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(yeast5)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.964
AUPR=  0.766


# YEAST 6

In [ ]:
from google.colab import files
import io
uploaded= files.upload()

Saving yeast6.csv to yeast6 (1).csv


In [ ]:
columns = ["Mcg","Gvh","Alm","Mit","Erl","Vac","class"]
yeast6=pd.read_csv(io.BytesIO(uploaded['yeast6.csv']),names=columns,header=None)
yeast6.head()

,,Mcg,Gvh,Alm,Mit,Erl,Vac,class
0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,negative
0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,negative
0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,negative
0.58,0.44,0.57,0.13,0.5,0.0,0.54,0.22,negative
0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,negative


In [ ]:
yeast6.reset_index(drop=True, inplace=True)
yeast6.head()

,Mcg,Gvh,Alm,Mit,Erl,Vac,class
0,0.47,0.13,0.5,0.0,0.48,0.22,negative
1,0.48,0.27,0.5,0.0,0.53,0.22,negative
2,0.49,0.15,0.5,0.0,0.53,0.22,negative
3,0.57,0.13,0.5,0.0,0.54,0.22,negative
4,0.48,0.54,0.5,0.0,0.48,0.22,negative


In [ ]:
lbl_encoder=LabelEncoder()
yeast6[['class']]=lbl_encoder.fit_transform(yeast6[['class']])
yeast6.head()

,Mcg,Gvh,Alm,Mit,Erl,Vac,class
0,0.47,0.13,0.5,0.0,0.48,0.22,0
1,0.48,0.27,0.5,0.0,0.53,0.22,0
2,0.49,0.15,0.5,0.0,0.53,0.22,0
3,0.57,0.13,0.5,0.0,0.54,0.22,0
4,0.48,0.54,0.5,0.0,0.48,0.22,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(yeast6)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.619
AUPR=  0.033


# ecoli-0-3-4 vs 5

In [ ]:
# Glass 6 data overview
from google.colab import files
import io
uploaded= files.upload()

Saving ecoli-0-3-4_vs_5.csv to ecoli-0-3-4_vs_5.csv


In [ ]:
columns = ["a1","a2","a3","a4","a5","a6","a7","class"]
ecoli=pd.read_csv(io.BytesIO(uploaded['ecoli-0-3-4_vs_5.csv']),names=columns,header=None)
ecoli.head()

,a1,a2,a3,a4,a5,a6,a7,class
0,49.0,29.0,48.0,5.0,56.0,24.0,35.0,negative
1,7.0,4.0,48.0,5.0,54.0,35.0,44.0,negative
2,56.0,4.0,48.0,5.0,49.0,37.0,46.0,negative
3,59.0,49.0,48.0,5.0,52.0,45.0,36.0,negative
4,23.0,32.0,48.0,5.0,55.0,25.0,35.0,negative


In [ ]:
lbl_encoder=LabelEncoder()
ecoli[['class']]=lbl_encoder.fit_transform(ecoli[['class']])
ecoli.head()

,a1,a2,a3,a4,a5,a6,a7,class
0,49.0,29.0,48.0,5.0,56.0,24.0,35.0,0
1,7.0,4.0,48.0,5.0,54.0,35.0,44.0,0
2,56.0,4.0,48.0,5.0,49.0,37.0,46.0,0
3,59.0,49.0,48.0,5.0,52.0,45.0,36.0,0
4,23.0,32.0,48.0,5.0,55.0,25.0,35.0,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(ecoli)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.926
AUPR=  0.461


# abalone19

In [ ]:
#Data overview
from google.colab import files
import io
uploaded= files.upload()

Saving abalone19.csv to abalone19.csv


In [ ]:
columns = ["Sex","Length","Diameter","Height","Whole_weight","Shucked_weight","Viscera_weight","Shell_weight","class"]
abalone=pd.read_csv(io.BytesIO(uploaded['abalone19.csv']),names=columns,header=None)
abalone.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,class
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,negative
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,negative
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,negative
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,negative
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,negative


In [ ]:
lbl_encoder=LabelEncoder()
abalone[['class']]=lbl_encoder.fit_transform(abalone[['class']])
abalone.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,class
0,M,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,1
1,M,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,1
2,F,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,1
3,M,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,1
4,I,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,1


In [ ]:
lbl_encoder=LabelEncoder()
abalone[['Sex']]=lbl_encoder.fit_transform(abalone[['Sex']])
abalone.head()

,Sex,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight,class
0,2,0.455,0.365,0.095,0.5140,0.2245,0.1010,0.150,1
1,2,0.350,0.265,0.090,0.2255,0.0995,0.0485,0.070,1
2,0,0.530,0.420,0.135,0.6770,0.2565,0.1415,0.210,1
3,2,0.440,0.365,0.125,0.5160,0.2155,0.1140,0.155,1
4,1,0.330,0.255,0.080,0.2050,0.0895,0.0395,0.055,1


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(abalone)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.543
AUPR=  0.993


# glass-0-1-4-6 vs 2

In [ ]:
from google.colab import files
import io
uploaded= files.upload()

Saving glass-0-1-4-6_vs_2.csv to glass-0-1-4-6_vs_2.csv


In [ ]:
columns = ["RI","Na","Mg","Al","Si","K","Ca","Ba","Fe","class"]
glass014=pd.read_csv(io.BytesIO(uploaded['glass-0-1-4-6_vs_2.csv']),names=columns,header=None)
glass014.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,negative
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,negative
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,negative
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,negative
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,negative


In [ ]:
lbl_encoder=LabelEncoder()
glass014[['class']]=lbl_encoder.fit_transform(glass014[['class']])
glass014.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,0
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,0
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,0
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,0
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(glass014)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.434
AUPR=  0.059


# glass2

In [ ]:
# Glass 2 data overview
from google.colab import files
import io
uploaded= files.upload()

Saving glass2.csv to glass2.csv


In [ ]:
columns = ["RI","Na","Mg","Al","Si","K","Ca","Ba","Fe","class"]
glass2=pd.read_csv(io.BytesIO(uploaded['glass2.csv']),names=columns,header=None)
glass2.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.51673,13.30,3.64,1.53,72.53,0.65,8.03,0.00,0.29,negative
1,1.51750,12.82,3.55,1.49,72.75,0.54,8.52,0.00,0.19,negative
2,1.51775,12.85,3.48,1.23,72.97,0.61,8.56,0.09,0.22,negative
3,1.51646,13.41,3.55,1.25,72.81,0.68,8.10,0.00,0.00,negative
4,1.51761,12.81,3.54,1.23,73.24,0.58,8.39,0.00,0.00,negative


In [ ]:
lbl_encoder=LabelEncoder()
glass2[['class']]=lbl_encoder.fit_transform(glass2[['class']])
glass2.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.51673,13.30,3.64,1.53,72.53,0.65,8.03,0.00,0.29,0
1,1.51750,12.82,3.55,1.49,72.75,0.54,8.52,0.00,0.19,0
2,1.51775,12.85,3.48,1.23,72.97,0.61,8.56,0.09,0.22,0
3,1.51646,13.41,3.55,1.25,72.81,0.68,8.10,0.00,0.00,0
4,1.51761,12.81,3.54,1.23,73.24,0.58,8.39,0.00,0.00,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(glass2)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.697
AUPR=  0.219


# page-blocks0

In [ ]:
from google.colab import files
import io
uploaded= files.upload()

Saving pageblocks.csv to pageblocks.csv


In [ ]:
columns = ["height","lenght","area","eccen","p_black","p_and","mean_tr","blackpix","blackand","wb_trans","class"]
page=pd.read_csv(io.BytesIO(uploaded['pageblocks.csv']),names=columns,header=None)
page.head()

,height,lenght,area,eccen,p_black,p_and,mean_tr,blackpix,blackand,wb_trans,class
0,5.0,7.0,35.0,1.400,0.486,0.914,8.50,17.0,32.0,2.0,1
1,9.0,37.0,333.0,4.111,0.532,0.913,3.22,177.0,304.0,55.0,1
2,2.0,18.0,36.0,9.000,0.667,0.694,24.00,24.0,25.0,1.0,2
3,9.0,34.0,306.0,3.778,0.320,0.886,1.56,98.0,271.0,63.0,1
4,11.0,99.0,1089.0,9.000,0.230,0.601,2.14,250.0,654.0,117.0,1


# glass6 

In [ ]:
# Glass 6 data overview
from google.colab import files
import io
uploaded= files.upload()


Saving glass6.csv to glass6.csv


In [ ]:
columns = ["RI","Na","Mg","Al","Si","K","Ca","Ba","Fe","class"]
glass6=pd.read_csv(io.BytesIO(uploaded['glass6.csv']),names=columns,header=None)
glass6.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,negative
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,negative
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,negative
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,negative
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,negative


In [ ]:
from sklearn.preprocessing import LabelEncoder
lbl_encoder=LabelEncoder()
glass6[['class']]=lbl_encoder.fit_transform(glass6[['class']])
glass6.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,0
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,0
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,0
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,0
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(glass6)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.935
AUPR=  0.708


# yeast-1 vs 7

In [ ]:
from google.colab import files
import io
uploaded= files.upload()

Saving yeast-1_vs_7.csv to yeast-1_vs_7.csv


In [ ]:
columns = ["Mcg","Gvh","Alm","Mit","Erl","Vac","class"]
yeast1=pd.read_csv(io.BytesIO(uploaded['yeast-1_vs_7.csv']),names=columns,header=None)
yeast1.head()

,Mcg,Gvh,Alm,Mit,Erl,Vac,class
0.58,0.44,0.57,0.13,0.5,0.54,0.22,negative
0.48,0.45,0.59,0.20,0.5,0.58,0.34,negative
0.43,0.39,0.54,0.21,0.5,0.53,0.27,negative
0.42,0.37,0.59,0.20,0.5,0.52,0.29,negative
0.58,0.47,0.54,0.11,0.5,0.51,0.26,negative


In [ ]:
yeast1.reset_index(drop=True, inplace=True)
yeast1.head()

,Mcg,Gvh,Alm,Mit,Erl,Vac,class
0,0.44,0.57,0.13,0.5,0.54,0.22,negative
1,0.45,0.59,0.20,0.5,0.58,0.34,negative
2,0.39,0.54,0.21,0.5,0.53,0.27,negative
3,0.37,0.59,0.20,0.5,0.52,0.29,negative
4,0.47,0.54,0.11,0.5,0.51,0.26,negative


In [ ]:
lbl_encoder=LabelEncoder()
yeast1[['class']]=lbl_encoder.fit_transform(yeast1[['class']])
yeast1.head()

,Mcg,Gvh,Alm,Mit,Erl,Vac,class
0,0.44,0.57,0.13,0.5,0.54,0.22,0
1,0.45,0.59,0.20,0.5,0.58,0.34,0
2,0.39,0.54,0.21,0.5,0.53,0.27,0
3,0.37,0.59,0.20,0.5,0.52,0.29,0
4,0.47,0.54,0.11,0.5,0.51,0.26,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(yeast1)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.843
AUPR=  0.412


# poker-8-9 vs 6

In [ ]:
from google.colab import files
import io
uploaded= files.upload()

Saving poker-8-9_vs_6.csv to poker-8-9_vs_6.csv


In [ ]:
columns = ["S1","C1","S2","C2","S3","C3","S4","C4","S5","C5","class"]
poker=pd.read_csv(io.BytesIO(uploaded['poker-8-9_vs_6.csv']),names=columns,header=None)
poker.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,class
0,4,5,1,9,3,5,4,9,2,9,negative
1,4,11,4,5,2,11,3,11,1,5,negative
2,3,1,4,1,3,4,1,4,2,1,negative
3,3,5,2,5,3,1,2,1,4,1,negative
4,4,13,2,13,4,7,3,7,1,13,negative


In [ ]:
lbl_encoder=LabelEncoder()
poker[['class']]=lbl_encoder.fit_transform(poker[['class']])
poker.head()

,S1,C1,S2,C2,S3,C3,S4,C4,S5,C5,class
0,4,5,1,9,3,5,4,9,2,9,0
1,4,11,4,5,2,11,3,11,1,5,0
2,3,1,4,1,3,4,1,4,2,1,0
3,3,5,2,5,3,1,2,1,4,1,0
4,4,13,2,13,4,7,3,7,1,13,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(poker)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.673
AUPR=  0.239


# haberman

In [ ]:
#Data overview
from google.colab import files
import io
uploaded= files.upload()

Saving haberman.csv to haberman.csv


In [ ]:
columns = ["Age","Year","Number","class"]
haberman=pd.read_csv(io.BytesIO(uploaded['haberman.csv']),names=columns,header=None)
haberman.head()

,Age,Year,Number,class
0,38,59,2,negative
1,39,63,4,negative
2,49,62,1,negative
3,53,60,2,negative
4,47,68,4,negative


In [ ]:
lbl_encoder=LabelEncoder()
haberman[['class']]=lbl_encoder.fit_transform(haberman[['class']])
haberman.head()

,Age,Year,Number,class
0,38,59,2,0
1,39,63,4,0
2,49,62,1,0
3,53,60,2,0
4,47,68,4,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(haberman)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.583
AUPR=  0.267


# winequality-red8 vs 6

In [ ]:
from google.colab import files
import io
uploaded= files.upload()

Saving winequality-red-8_vs_6.csv to winequality-red-8_vs_6.csv


In [ ]:
columns = ["FixedAcidity","VolatileAcidity","CitricAcid","ResidualSugar","Chlorides","FreeSulfurDioxide","TotalSulfurDioxide","Density","PH","Sulphates","Alcohol","class"]
wine=pd.read_csv(io.BytesIO(uploaded['winequality-red-8_vs_6.csv']),names=columns,header=None)
wine.head()

,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,PH,Sulphates,Alcohol,class
0,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,negative
1,7.9,0.320,0.51,1.8,0.341,17.0,56.0,0.9969,3.04,1.08,9.2,negative
2,8.9,0.220,0.48,1.8,0.077,29.0,60.0,0.9968,3.39,0.53,9.4,negative
3,6.9,0.400,0.14,2.4,0.085,21.0,40.0,0.9968,3.43,0.63,9.7,negative
4,7.8,0.645,0.00,2.0,0.082,8.0,16.0,0.9964,3.38,0.59,9.8,negative


In [ ]:
lbl_encoder=LabelEncoder()
wine[['class']]=lbl_encoder.fit_transform(wine[['class']])
wine.head()

,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,PH,Sulphates,Alcohol,class
0,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,0
1,7.9,0.320,0.51,1.8,0.341,17.0,56.0,0.9969,3.04,1.08,9.2,0
2,8.9,0.220,0.48,1.8,0.077,29.0,60.0,0.9968,3.39,0.53,9.4,0
3,6.9,0.400,0.14,2.4,0.085,21.0,40.0,0.9968,3.43,0.63,9.7,0
4,7.8,0.645,0.00,2.0,0.082,8.0,16.0,0.9964,3.38,0.59,9.8,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(wine)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.695
AUPR=  0.15


# led7digit-0-2-4-5-6-7-8-9 vs 1


In [ ]:
from google.colab import files
import io
uploaded= files.upload()

Saving led7digit-0-2-4-5-6-7-8-9_vs_1.csv to led7digit-0-2-4-5-6-7-8-9_vs_1.csv


In [ ]:
columns = ["Led1","Led2","Led3","Led4","Led5","Led6","Led7","class"]
led7=pd.read_csv(io.BytesIO(uploaded['led7digit-0-2-4-5-6-7-8-9_vs_1.csv']),names=columns,header=None)
led7.head()

,Led1,Led2,Led3,Led4,Led5,Led6,Led7,class
0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,negative
1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,negative
2,1.0,1.0,1.0,0.0,1.0,1.0,1.0,negative
3,1.0,1.0,1.0,0.0,1.0,1.0,1.0,negative
4,1.0,1.0,1.0,0.0,1.0,1.0,1.0,negative


In [ ]:
lbl_encoder=LabelEncoder()
led7[['class']]=lbl_encoder.fit_transform(led7[['class']])
led7.head()

,Led1,Led2,Led3,Led4,Led5,Led6,Led7,class
0,0.0,1.0,1.0,0.0,1.0,1.0,1.0,0
1,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0
2,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0
3,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0
4,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(led7)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.914
AUPR=  0.776


# glass0

In [ ]:
# Glass 0 data overview
from google.colab import files
import io
uploaded= files.upload()

Saving glass0.csv to glass0.csv


In [ ]:
columns = ["RI","Na","Mg","Al","Si","K","Ca","Ba","Fe","class"]
glass0=pd.read_csv(io.BytesIO(uploaded['glass0.csv']),names=columns,header=None)
glass0.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,positive
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,positive
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,positive
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,positive
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,positive


In [ ]:
lbl_encoder=LabelEncoder()
glass0[['class']]=lbl_encoder.fit_transform(glass0[['class']])
glass0.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,1
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,1
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,1
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,1
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,1


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(glass0)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.811
AUPR=  0.571


# glass-0-1-5 vs 2

In [ ]:
# Glass 6 data overview
from google.colab import files
import io
uploaded= files.upload()


Saving glass-0-1-5_vs_2.csv to glass-0-1-5_vs_2.csv


In [ ]:
columns = ["RI","Na","Mg","Al","Si","K","Ca","Ba","Fe","class"]
glass=pd.read_csv(io.BytesIO(uploaded['glass-0-1-5_vs_2.csv']),names=columns,header=None)
glass.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,negative
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,negative
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,negative
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,negative
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,negative


In [ ]:
lbl_encoder=LabelEncoder()
glass[['class']]=lbl_encoder.fit_transform(glass[['class']])
glass.head()

,RI,Na,Mg,Al,Si,K,Ca,Ba,Fe,class
0,1.515888,12.87795,3.43036,1.40066,73.2820,0.68931,8.04468,0.0,0.1224,0
1,1.517642,12.97770,3.53812,1.21127,73.0020,0.65205,8.52888,0.0,0.0000,0
2,1.522130,14.20795,3.82099,0.46976,71.7700,0.11178,9.57260,0.0,0.0000,0
3,1.522221,13.21045,3.77160,0.79076,71.9884,0.13041,10.24520,0.0,0.0000,0
4,1.517551,13.39000,3.65935,1.18880,72.7892,0.57132,8.27064,0.0,0.0561,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(glass)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.886
AUPR=  0.626


# yeast-0-2-5-7-9 vs 3-6-8


In [ ]:
from google.colab import files
import io
uploaded= files.upload()

Saving yeast-0-2-5-7-9_vs_3-6-8.csv to yeast-0-2-5-7-9_vs_3-6-8.csv


In [ ]:
columns = ["Mcg","Gvh","Alm","Mit","Erl","Pox","Vac","Nuc","class"]
yeast0=pd.read_csv(io.BytesIO(uploaded['yeast-0-2-5-7-9_vs_3-6-8.csv']),names=columns,header=None)
yeast0.head()

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,class
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,negative
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,negative
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,negative
3,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,negative
4,0.50,0.54,0.48,0.65,0.5,0.0,0.53,0.22,negative


In [ ]:
lbl_encoder=LabelEncoder()
yeast0[['class']]=lbl_encoder.fit_transform(yeast0[['class']])
yeast0.head()

,Mcg,Gvh,Alm,Mit,Erl,Pox,Vac,Nuc,class
0,0.58,0.61,0.47,0.13,0.5,0.0,0.48,0.22,0
1,0.43,0.67,0.48,0.27,0.5,0.0,0.53,0.22,0
2,0.64,0.62,0.49,0.15,0.5,0.0,0.53,0.22,0
3,0.42,0.44,0.48,0.54,0.5,0.0,0.48,0.22,0
4,0.50,0.54,0.48,0.65,0.5,0.0,0.53,0.22,0


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(yeast0)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.949
AUPR=  0.558


# pima

In [ ]:
#Data overview
from google.colab import files
import io
uploaded= files.upload()

Saving pima.csv to pima.csv


In [ ]:
columns = ["Preg","Plas","Pres","Skin","Insu","Mass","Pedi","Age","class"]
pima=pd.read_csv(io.BytesIO(uploaded['pima.csv']),names=columns,header=None)
pima.head()

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,class
0,6,148,72,35,0,33.6,0.627,50,positive
1,1,85,66,29,0,26.6,0.351,31,negative
2,8,183,64,0,0,23.3,0.672,32,positive
3,1,89,66,23,94,28.1,0.167,21,negative
4,0,137,40,35,168,43.1,2.288,33,positive


In [ ]:
lbl_encoder=LabelEncoder()
pima[['class']]=lbl_encoder.fit_transform(pima[['class']])
pima.head()

,Preg,Plas,Pres,Skin,Insu,Mass,Pedi,Age,class
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [ ]:
perf1=[]
perf2=[]
for i in range(30):
    auc,aupr=Score_Algo(pima)
    perf1.append(auc)
    perf2.append(aupr)
print('AUC= ',round(np.mean(perf1),3))
print('AUPR= ',round(np.mean(perf2),3))

AUC=  0.736
AUPR=  0.524
